# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Aim-of-notebook" data-toc-modified-id="Aim-of-notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Aim of notebook</a></div><div class="lev1 toc-item"><a href="#Load-the-main-airport-traffic-data" data-toc-modified-id="Load-the-main-airport-traffic-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the main airport traffic data</a></div><div class="lev1 toc-item"><a href="#Load-lookup-table-provided-by-BTS" data-toc-modified-id="Load-lookup-table-provided-by-BTS-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load lookup table provided by BTS</a></div><div class="lev1 toc-item"><a href="#Create-enhanced-lookuptable" data-toc-modified-id="Create-enhanced-lookuptable-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create <em>enhanced</em> lookuptable</a></div><div class="lev2 toc-item"><a href="#Remove-Code-that-is-not-present-our-dataset" data-toc-modified-id="Remove-Code-that-is-not-present-our-dataset-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Remove Code that is not present our dataset</a></div><div class="lev2 toc-item"><a href="#Parse-state,city,-and-airport-name-from-'Description'-field" data-toc-modified-id="Parse-state,city,-and-airport-name-from-'Description'-field-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Parse state,city, and airport-name from 'Description' field</a></div><div class="lev2 toc-item"><a href="#Add-state-'region'-information" data-toc-modified-id="Add-state-'region'-information-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Add state 'region' information</a></div><div class="lev2 toc-item"><a href="#Add-airport-latitude-and-longitude-information-using-geocoder" data-toc-modified-id="Add-airport-latitude-and-longitude-information-using-geocoder-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Add airport latitude and longitude information using geocoder</a></div><div class="lev2 toc-item"><a href="#Add-column-with-both-city-and-state" data-toc-modified-id="Add-column-with-both-city-and-state-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Add column with both city and state</a></div><div class="lev1 toc-item"><a href="#All-done.-Save-dataframe-on-disk" data-toc-modified-id="All-done.-Save-dataframe-on-disk-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>All done. Save dataframe on disk</a></div>

In [1]:
%matplotlib inline
import pandas as pd
import time
import numpy as np

from pprint import pprint
from util import print_time

# Aim of notebook

- The [airport traffic dataset](http://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time) encodes the airport with unique ID numbers.

- In this notebook, we'll create an *enhanced lookup-table* by taking the lookup table provided by the BTS ([download link](http://www.transtats.bts.gov/Download_Lookup.asp?Lookup=L_AIRPORT_ID)) and adding additional relevant information regarding the airport (such as latitude/longitutde info)

# Load the main airport traffic data

- Load 3 years worth of air-traffic data provided by BTS ([link](http://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time))
- (from November 2013 to October 2016)

In [2]:
from util import load_airport_data_3years
df_data = load_airport_data_3years()

 ... load dataframe from 2013-11.zip 
 ... load dataframe from 2013-12.zip 
 ... load dataframe from 2014-01.zip 
 ... load dataframe from 2014-02.zip 
 ... load dataframe from 2014-03.zip 
 ... load dataframe from 2014-04.zip 
 ... load dataframe from 2014-05.zip 
 ... load dataframe from 2014-06.zip 
 ... load dataframe from 2014-07.zip 
 ... load dataframe from 2014-08.zip 
 ... load dataframe from 2014-09.zip 
 ... load dataframe from 2014-10.zip 
 ... load dataframe from 2014-11.zip 
 ... load dataframe from 2014-12.zip 
 ... load dataframe from 2015-01.zip 
 ... load dataframe from 2015-02.zip 
 ... load dataframe from 2015-03.zip 
 ... load dataframe from 2015-04.zip 
 ... load dataframe from 2015-05.zip 
 ... load dataframe from 2015-06.zip 
 ... load dataframe from 2015-07.zip 
 ... load dataframe from 2015-08.zip 
 ... load dataframe from 2015-09.zip 
 ... load dataframe from 2015-10.zip 
 ... load dataframe from 2015-11.zip 
 ... load dataframe from 2015-12.zip 
 ... load da

In [3]:
print df_data.shape
df_data.head(n=5)

(17364696, 7)


,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID
0,2013,4,11,3,7,12478,10693
1,2013,4,11,4,1,12478,10693
2,2013,4,11,5,2,12478,10693
3,2013,4,11,6,3,12478,10693
4,2013,4,11,7,4,12478,10693


# Load lookup table provided by BTS

In [4]:
df_lookup = pd.read_csv('../data/L_AIRPORT_ID.csv')
print df_lookup.shape

(6409, 2)


# Create *enhanced* lookuptable

In [5]:
df_lookup.head(n=10)

,Code,Description
0,10001,"Afognak Lake, AK: Afognak Lake Airport"
1,10003,"Granite Mountain, AK: Bear Creek Mining Strip"
2,10004,"Lik, AK: Lik Mining Camp"
3,10005,"Little Squaw, AK: Little Squaw Airport"
4,10006,"Kizhuyak, AK: Kizhuyak Bay"
5,10007,"Klawock, AK: Klawock Seaplane Base"
6,10008,"Elizabeth Island, AK: Elizabeth Island Airport"
7,10009,"Homer, AK: Augustin Island"
8,10010,"Hudson, NY: Columbia County"
9,10011,"Peach Springs, AZ: Grand Canyon West"


## Remove Code that is not present our dataset

In [6]:
# unique ID's in the dataset
uniq_orig = df_data['ORIGIN_AIRPORT_ID'].unique().tolist() 
uniq_dest = df_data['DEST_AIRPORT_ID'].unique().tolist()

# apply ``set`` function to get unique items in concatenated list
uniq_id = list(set(uniq_orig + uniq_dest))

print "There are {} Airport-Codes in the lookup table".format(df_lookup.shape[0])
print "There are {} unique airport-codes in our dataset".format(uniq_id.__len__())

There are 6409 Airport-Codes in the lookup table
There are 334 unique airport-codes in our dataset


Let's filter/drop the rows/records that we do not need in our analysis

In [7]:
# only keep the items in the main dataframe
_mask = df_lookup['Code'].isin( uniq_id )
df_lookup = df_lookup[ _mask ].reset_index(drop=True)

print df_lookup.shape
df_lookup.head(10)

(334, 2)


,Code,Description
0,10135,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ..."
1,10136,"Abilene, TX: Abilene Regional"
2,10140,"Albuquerque, NM: Albuquerque International Sun..."
3,10141,"Aberdeen, SD: Aberdeen Regional"
4,10146,"Albany, GA: Southwest Georgia Regional"
5,10154,"Nantucket, MA: Nantucket Memorial"
6,10155,"Waco, TX: Waco Regional"
7,10157,"Arcata/Eureka, CA: Arcata"
8,10158,"Atlantic City, NJ: Atlantic City International"
9,10165,"Adak Island, AK: Adak"


## Parse state,city, and airport-name from 'Description' field

- Above we realize that the ``Description`` field contains information regarding the *city*, *state*, and *name* of the airport.

- Let's create individual field for each information.

- Fortunately, the ``Description`` column uses a comma (``,``) and colon (``:``) to delimit the City, State, Airport-name information, so splitting these are is straightforward.



In [8]:
# apply string "split" method to break information up
df_parse = map(lambda splits: {'City':splits[0],'State':splits[2],'Airport':splits[4]},
               df_lookup['Description'].str.split(r'(,\s|:\s)') )

pprint(df_parse[:5])

# convert dict to dataframe
df_parse = pd.DataFrame(df_parse)
df_parse.head(5)

[{'Airport': 'Lehigh Valley International',
  'City': 'Allentown/Bethlehem/Easton',
  'State': 'PA'},
 {'Airport': 'Abilene Regional', 'City': 'Abilene', 'State': 'TX'},
 {'Airport': 'Albuquerque International Sunport',
  'City': 'Albuquerque',
  'State': 'NM'},
 {'Airport': 'Aberdeen Regional', 'City': 'Aberdeen', 'State': 'SD'},
 {'Airport': 'Southwest Georgia Regional', 'City': 'Albany', 'State': 'GA'}]


,Airport,City,State
0,Lehigh Valley International,Allentown/Bethlehem/Easton,PA
1,Abilene Regional,Abilene,TX
2,Albuquerque International Sunport,Albuquerque,NM
3,Aberdeen Regional,Aberdeen,SD
4,Southwest Georgia Regional,Albany,GA


In [9]:
# now we can readily add these information to our lookup table
df_lookup = df_lookup.join(df_parse)

print df_lookup.shape
df_lookup.head()

(334, 5)


,Code,Description,Airport,City,State
0,10135,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",Lehigh Valley International,Allentown/Bethlehem/Easton,PA
1,10136,"Abilene, TX: Abilene Regional",Abilene Regional,Abilene,TX
2,10140,"Albuquerque, NM: Albuquerque International Sun...",Albuquerque International Sunport,Albuquerque,NM
3,10141,"Aberdeen, SD: Aberdeen Regional",Aberdeen Regional,Aberdeen,SD
4,10146,"Albany, GA: Southwest Georgia Regional",Southwest Georgia Regional,Albany,GA


## Add state 'region' information

I also would like to study patterns among the four-regions in the United States: 

(1) Northeast
(2) South
(3) West
(4) Midwest

I saved a json lookup file for this purpose

In [10]:
%%bash
cat ../data/us_states_regions.json

{
"Northeast" : ["Connecticut","Maine", "Massachusetts", "New Hampshire", "Rhode Island", "Vermont","New Jersey", "New York", "Pennsylvania"],
"Midwest"   : ["Illinois", "Indiana", "Michigan", "Ohio", "Wisconsin", "Iowa", "Kansas", "Minnesota", "Missouri", "Nebraska", "North Dakota", "South Dakota"],
"South"     : [ "Delaware", "Florida", "Georgia", "Maryland", "North Carolina", "South Carolina", "Virginia", "District of Columbia", "West Virginia",             "Alabama", "Kentucky", "Mississippi", "Tennessee","Arkansas", "Louisiana", "Oklahoma", "Texas"],
"West"      : ["Arizona", "Colorado", "Idaho", "Montana", "Nevada", "New Mexico", "Utah",  "Wyoming", "Alaska", "California", "Hawaii", "Oregon", "Washington"]
}

In [11]:
import json
with open('../data/us_states_regions.json','r') as f:
    regions = json.load(f)

print regions.keys()
print regions.values()

[u'West', u'Northeast', u'Midwest', u'South']
[[u'Arizona', u'Colorado', u'Idaho', u'Montana', u'Nevada', u'New Mexico', u'Utah', u'Wyoming', u'Alaska', u'California', u'Hawaii', u'Oregon', u'Washington'], [u'Connecticut', u'Maine', u'Massachusetts', u'New Hampshire', u'Rhode Island', u'Vermont', u'New Jersey', u'New York', u'Pennsylvania'], [u'Illinois', u'Indiana', u'Michigan', u'Ohio', u'Wisconsin', u'Iowa', u'Kansas', u'Minnesota', u'Missouri', u'Nebraska', u'North Dakota', u'South Dakota'], [u'Delaware', u'Florida', u'Georgia', u'Maryland', u'North Carolina', u'South Carolina', u'Virginia', u'District of Columbia', u'West Virginia', u'Alabama', u'Kentucky', u'Mississippi', u'Tennessee', u'Arkansas', u'Louisiana', u'Oklahoma', u'Texas']]


In [12]:
df_region = []
for key in regions:
    _dftmp = pd.DataFrame( regions[key], columns=['State']  )
    _dftmp['Region'] = key
    df_region.append(_dftmp)
    
df_region = pd.concat(df_region,ignore_index=True)

df_region.head()

,State,Region
0,Arizona,West
1,Colorado,West
2,Idaho,West
3,Montana,West
4,Nevada,West


Let's use a hash-table (source) to map state name to its abbreviation

In [13]:
from util import hash_state_to_abbrev
hash_state = hash_state_to_abbrev()

df_region['State'] = df_region['State'].map(lambda key: hash_state[key])
df_region.head()

,State,Region
0,AZ,West
1,CO,West
2,ID,West
3,MT,West
4,NV,West


In [14]:
# good, we're now ready to join this "Region" information to our lookup table
df_lookup = df_lookup.merge(df_region,on='State',how='left')

df_lookup.head(10)

,Code,Description,Airport,City,State,Region
0,10135,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",Lehigh Valley International,Allentown/Bethlehem/Easton,PA,Northeast
1,10136,"Abilene, TX: Abilene Regional",Abilene Regional,Abilene,TX,South
2,10140,"Albuquerque, NM: Albuquerque International Sun...",Albuquerque International Sunport,Albuquerque,NM,West
3,10141,"Aberdeen, SD: Aberdeen Regional",Aberdeen Regional,Aberdeen,SD,Midwest
4,10146,"Albany, GA: Southwest Georgia Regional",Southwest Georgia Regional,Albany,GA,South
5,10154,"Nantucket, MA: Nantucket Memorial",Nantucket Memorial,Nantucket,MA,Northeast
6,10155,"Waco, TX: Waco Regional",Waco Regional,Waco,TX,South
7,10157,"Arcata/Eureka, CA: Arcata",Arcata,Arcata/Eureka,CA,West
8,10158,"Atlantic City, NJ: Atlantic City International",Atlantic City International,Atlantic City,NJ,Northeast
9,10165,"Adak Island, AK: Adak",Adak,Adak Island,AK,West


## Add airport latitude and longitude information using geocoder

Next we'll query the geograhical location of each airport using geocoders from ``geopy`` ([link](https://geopy.readthedocs.io/en/1.10.0/)).

This information will be useful especially when creating visualization plots.

The cell below is going to take a while, so good time to brew a coffee... (need to add breaks between API requests to avoid getting timed-out)

In [15]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

t = time.time()
lat,lon= [],[]
n_items = df_lookup.shape[0]
for i,airport in enumerate(df_lookup['Airport']):
    if i%20==0:
         print '({:3} out of {})'.format(i,n_items),print_time(t)

    loc = geolocator.geocode(airport)
    time.sleep(10) # add break to avoid api service timeouts

    if loc is not None:
        lat.append(loc[1][0])
        lon.append(loc[1][1])        
    else:
        print '    lookup failed for: ' + airport
        lon.append(None)
        lat.append(None)



(  0 out of 334) Elapsed time:  0.00 seconds
( 20 out of 334) Elapsed time: 209.90 seconds
    lookup failed for: Western Neb. Regional/William B. Heilig Field
    lookup failed for: Greater Binghamton/Edwin A. Link Field
( 40 out of 334) Elapsed time: 418.37 seconds
    lookup failed for: Boise Air Terminal
    lookup failed for: Brownsville South Padre Island International
    lookup failed for: Baltimore/Washington International Thurgood Marshall
    lookup failed for: Akron-Canton Regional
( 60 out of 334) Elapsed time: 627.09 seconds
    lookup failed for: Charleston AFB/International
    lookup failed for: Casper/Natrona County International
( 80 out of 334) Elapsed time: 835.74 seconds
    lookup failed for: Dickinson - Theodore Roosevelt Regional
(100 out of 334) Elapsed time: 1044.65 seconds
    lookup failed for: Northwest Florida Beaches International
    lookup failed for: Erie International/Tom Ridge Field
(120 out of 334) Elapsed time: 1253.41 seconds
    lookup failed fo

In [16]:
# add as new columns
df_lookup['lat'] = lat
df_lookup['lon'] = lon

n_nans = df_lookup['lat'].isnull().sum(axis=0)
print "-- {} NANs out {} ({:.2f}%) --".format(n_nans,n_items,n_nans/float(n_items)*100)

-- 36 NANs out 334 (10.78%) --


- So some lookup failed...but most succeeded

- For airports that failed, search using City + State information (lose locality a bit but will suffice for our analysis)

In [17]:
idx_nan = [] # keep track of the index location that may fail yet again
for i in xrange(n_items):
    print i,
    if lat[i] is not None:
        continue
    city,state = df_lookup['City'].ix[i], df_lookup['State'].ix[i]
    loc = geolocator.geocode(city+', '+state)
    time.sleep(10) # add break to avoid api service timeouts

    if loc is not None:
        lat[i],lon[i] = loc[1]
    else:
        print '    lookup failed for: {}, {}'.format(city,state)
        idx_nan.append(i)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248     lookup failed for: Newport News/Williamsburg, VA
249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 2

In [18]:
# update columns
df_lookup['lat'] = lat
df_lookup['lon'] = lon

n_nans = df_lookup['lat'].isnull().sum(axis=0)
print "-- {} NANs out {} ({:.2f}%) --".format(n_nans,n_items,n_nans/float(n_items)*100)

-- 2 NANs out 334 (0.60%) --


In [19]:
df_lookup = pd.read_csv('df_lookup_tmp2.csv')

IOError: File df_lookup_tmp2.csv does not exist

- So at this point, we have two lookup failures

- Although unelegant, I'll just manually query these in the geocoder

In [ ]:
print idx_nan
df_lookup[df_lookup['lat'].isnull()]

In [ ]:
lat,lon = geolocator.geocode('Newport News, VA')[1]
df_lookup['lat'].ix[248] = lat
df_lookup['lon'].ix[248] = lon

lat,lon = geolocator.geocode('Saipan')[1]
df_lookup['lat'].ix[302] = lat
df_lookup['lon'].ix[302] = lon

print df_lookup.isnull().sum()

## Add column with both city and state

- Since I am not familiar with many names of the airport, I'd rather work with City and State names.

- However, there may be multiple airports in the same city (eg, JKF and Laguardia in NYC), so uniqueness of "City/State" is not guaranteed.

- Here, I'll create yet another (and final) column containing both the City and State information, and modify duplicates as needed.



In [ ]:
df_lookup['City_State'] = df_lookup['City'] + ' (' + df_lookup['State'] + ')'

df_lookup.sample(5)

In [ ]:
# check duplicates in "City_State"
dups = df_lookup['City_State'].value_counts()
dups = dups[dups != 1]

dups

In [ ]:
# create hash-table for airport name lookup
hash_airport = df_lookup.set_index('Code')['Airport'].to_dict()
pprint({k: hash_airport[k] for k in hash_airport.keys()[:5]})

In [ ]:
# check duplicates in "City_State"
for dup in dups.index:
    print dup
    for i in df_lookup[df_lookup['City_State'] == dup].Code:
        print "    (Code = {:6}) Airport = {}".format((df_data['ORIGIN_AIRPORT_ID'] == i).sum(), hash_airport[i])

Again, kinda hacky, but will create manual replacement on these duplicates using the "cleaner" below

In [ ]:
cleaner = [
    ('Houston (TX) [Ell]', 'Ellington'), 
    ('Houston (TX) [WP.Hobby]', 'William P Hobby'), 
    ('Houston (TX) [G.Bush]',  'George Bush Intercontinental/Houston'), 
    ('Chicago (IL) [Midway]',   'Chicago Midway International'),
    ("Chicago (IL) [O'Hare]",   "Chicago O'Hare International"),
    ('Washington (DC) [R.Reagan]',   'Ronald Reagan Washington National'),
    ('Washington (DC) [W.Dulles]',   'Washington Dulles International'),
    ("New York (NY) [JFK]",   "John F. Kennedy International"),
    ("New York (NY) [Lag]",   "LaGuardia"),
]

for _replace, _airport in cleaner:
    df_lookup.loc[df_lookup['Airport'] == _airport, 'City_State'] = _replace

# check duplicates are removed
assert np.all(df_lookup['City_State'].value_counts() == 1)

# All done. Save dataframe on disk

- We have created our *enhanced* lookup table.

- Let's save this on disk for later analysis.

In [ ]:
print df_lookup.shape
df_lookup.sample(10).sort_index()

In [ ]:
df_lookup.to_csv('df_lookup.csv',index=False)